Mistral 7B on Google GCP and DataRobot
======================================

There are a wide variety of open source models. For example, there has been a lot of interest in LLama and variations such as Alpaca or Vicuna, Falcon, Mistral etc. Hosting these is a challenge as they require GPUs which are expensive so often customers want to compare cloud providers to find the best hosting option to meet their own needs. In this example we will work with Google Cloud Platform.

In addition, customers may want to integrate with the same cloud provider that hosts their VPC. That way they can ensure proper authentication and access only from within their VPC. While this authenticator uses authentication over the public internet, it should then be possible for the user to extend to leverage Google's cloud infrastructure to adjust to suit their cloud architectural needs, including provisioning scale out policies.

Finally, by leveraging Vertex AI in a managed format, it can integrate into the customer's existing infrastructure level monitoring needs. For example, instances can be labelled to correspond to the customer's billing attribution polices, or logs and analytics can be set up to be written into their Big Query for monitoring and analytics. 

Mistral 7B
========

For information about Mistral you can read the model card on HuggingFace [https://huggingface.co/mistralai/Mistral-7B-v0.1], the Arxiv page [https://arxiv.org/abs/2310.06825] and the release anouncement [https://mistral.ai/news/announcing-mistral-7b/]. It is available under an Apache 2.0 License [https://www.apache.org/licenses/LICENSE-2.0]

It features
* Grouped-Query Attention
* Rolling Buffer cache
* Pre-fill and Chunking
* Sliding-Window Attention
* Byte-fallback BPE tokenizer

the standard model is designed around text completion. It is able to fit on a single A10G instance.

Some data about the model:
Parameter Value
dim 4096
n_layers 32
head_dim 128
hidden_dim 14336
n_heads 32
n_kv_heads 8
window_size 4096
context_len 8192
vocab_size 32000

Mistral 7B-Instruct
===============

This is designed to follow user instructions by fine-tuning on instruction datasets available on HuggingFace. As part of this it was trained to use `[INST]` and `[/INST]` controls tokens around user messages as well as begin of system id `<s>`. For example:

* "`<s>`[INST] What is your favourite condiment? [/INST]"

Overview of GCP
===============
    
GCP instance types that can host Mistral with acceleration

* g2-standard-8 with 1 L4 GPU: 8 vCPUs, 32 GB of RAM, \$.85 ph + 64 GB (\$623 per month)
* n1-standard-16 with 2 V100 GPUs: 16 vCPUs, 60GB of RAM, \$.76 ph + 32 GB (\$388 per month)
* n1-standard-16 with 2 T4 GPUs: 16 vCPUS, 60GB of RAM + 32 GB + 32 GB (\$388 per month)
* a2-highgpu-1g with 1 A100 GPU: 12 vCPUs, 85GB of RAM, \$3.7 ph with  40GB (\$2,682 per month)

## 1. GCP Projects

Everything in GCP is owned by a project, which tracks billing, authentication, access control, etc. Whenever you interact in either the GUI or using the API clients you will need to be in a project context. You can create a project at [https://console.cloud.google.com/projectcreate] or under IAM & Admin > Create a Project.

## 2. Authentication and Service Accounts

GCP does *not* provide "API Keys". Instead it provides auto-expiring dynamic tokens after authorizing. Each separate request you make to Google will have a different token in the headers. After a period of time you will have to reauthorize. If you use Collab, Workbench, or Cloud Shell, Google will hande the authentication in the background for you after authorization. 

In order for our envionsed workflow to work, we will need a service account, an account which will run the cloud function workflow on our behalf (in this situation our account will be the principal). Using a service account, as opposed to a user account, means that multiple people can use our flow. In addition, there are certain things within GCP that can only be done by service accounts.

This service account will need access to the following roles:

* Vertex AI User

As well as the following permissions on our Cloud Stoage bucket to be able to write to it.
* Storage Legacy Bucket Owner
* Storage Legacy Object Owner

For more information about service accounts look here: [https://cloud.google.com/iam/docs/service-accounts-create].

## 3. Regions

We will use us-central1 (Iowa) for everything. This is because it is one of the two regions that have extensive GPU capacity (along with eu-west4 (Netherlands). The instance types available within Vertex AI vary by region.


## 4. Cloud Storage Bucket

Lastly, before getting started we will need a bucket to hold stuff as we work in GCP. Google in general requires a bucket to be able to execute many of the tasks since they require storing some information somewhere, and a bucket is where that happens

Once you have decided on a project, location, bucket and storage account fill in the values below:
                                                                                                                            

In [ ]:
import datetime

# Cloud project id.
PROJECT_ID = "octo-385122"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://octo-ephermal-storage"  # @param {type:"string"}

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = (
    "octo-experiments-service-accou@octo-385122.iam.gserviceaccount.com"  # @param {type:"string"}
)

# GCP Tags
# Fill in this dictionary with the required values to ensure the GCP resources are tagged correctly.
# These will be applied as labels to the resources created
GCP_TAGS = {
    # e.g "contact": "foo"
    #   "contact": "100007",
    #    "cost-center": "octo",
    #    "label": "genai",
    #    "environment": "dev",
    #    "project": "genai",
    "expiration": (datetime.datetime.utcnow() + datetime.timedelta(days=2)).strftime(
        "%Y-%m-%d"
    )  # t%H:%M:%Sz") # deault liftetime is up to 48H
}

First time setup 
================

The following sections consist of the tasks you should do once the first time to setup the enviornment and then you will need to restart the kernel. We will do the following steps:

1. Install the lastest version google cloud ai platform SDK.

This is the latest version that interacts with VertexAI

2. Set up authentication by using application default credenitals created in a different environment

The typical authoriztion flow is command line command --> web browser authorization --> enter authorization information. In our notebooks that is tricky as we would need to be able to enter the authorization into the output shell. We also typically would generate both cli credentials via `gcloud init` or `gcloud auth login` which would create credentials in a local credentials db as well as create application default credentials via `gcloud auth application-default login` to generate the credentials usable via the python library. 

Instead the flow will be: generate credentials on your local machine --> base64 encode --> update the environment variables to point to the uploaded credentials. These will be stored [https://docs.datarobot.com/en/docs/dr-notebooks/code-nb/dr-env-nb.html#environment-variables encrypted by DataRobot] but keep in mind that base64 encoding itself isn't encryption and treat these base64 encoded credentials like you would treat any password.

1. Create local credentials

Install the google sdk in your local enivornment. Then run `gcloud init` and make sure to set your default project accordingly. Then you can run `gcloud auth application-default login`

To install the google command line sdk in your local environment you can run:

```
curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-cli-445.0.0-linux-x86_64.tar.gz
tar -xf "google-cloud-cli-445.0.0-linux-x86_64.tar.gz"
./google-cloud-sdk/install.sh --usage-reporting=false --quiet
```

You should see output like:
```
Credentials saved to file: [/Users/mark/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "octo-385122" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.
```

2. Base64 credentials

The recommended version is to base64 encode as an environment variable. This is because any credentials uploaded to file storage is immediately shared when sharing the notebook. This can leverage DataRobot's built in credential storage.

```
base64 -i /Users/mark/.config/gcloud/application_default_credentials.json
```

to generate the string. You can then copy and paste this as the environment variable: `GOOGLE_ENCODED_CREDENTIAL`

3. Create credentials object

If you base64 encoded you can then create a credentials object in your local notebook environment via running the following:

```
import base64
import os
import google.oauth2.credential

adc_decoded = base64.b64decode(os.getenv("GOOGLE_ENCODED_CREDENTIAL"))
credentials = google.oauth2.credentials.Credentials.from_authorized_user_info(json.loads(adc_decoded))
```


Now we can pass the credentials into `aiplatfrom.init` to set the global configuration which will be used in future calls to Vertex AI

```
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET, credentials=credentials)
```

The following two cells will install the required packages and initialize the environment while the third cell will print out the current settings for user verification


In [ ]:
!pip -q install --upgrade google-cloud-aiplatform
!pip -q install requests
!pip -q install datarobot-early-access

In [ ]:
import base64
import json
import os

import google.auth
from google.cloud import aiplatform
import google.oauth2.credentials

adc_decoded = base64.b64decode(os.getenv("GOOGLE_ENCODED_CREDENTIAL"))
assert adc_decoded is not None
credentials = google.oauth2.credentials.Credentials.from_authorized_user_info(
    json.loads(adc_decoded)
)


# Bucket for storing intermediate stuff
STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")

# Initialize the ai platform global configuration for future calls
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET,
    service_account=SERVICE_ACCOUNT,
    credentials=credentials,
)

In [ ]:
# We can verify that the global configuation is set correctly
from google.cloud.aiplatform import initializer

print(f"The current project is: {initializer.global_config.project}")
print(f"The current location is: {initializer.global_config.location}")
print(f"The current GCP bucket is: {initializer.global_config.staging_bucket}")
print(f"The current default service account is: {initializer.global_config.service_account}")

The current project is: octo-385122
The current location is: us-central1
The current GCP bucket is: gs://octo-ephermal-storage/temporal
The current default service account is: octo-experiments-service-accou@octo-385122.iam.gserviceaccount.com


To deploy the model we will first upload it into Vertex's Model Registry and then we can deploy or undeploy from the endpoint where the actual provisioning of the instance happens. In Vertex AI the resources are set on the endpoint, not the model.

Model Serving with pytorch and vLLM
=======================

Vertex AI models can work with any docker container that provides an http endpoint for Vertex to pass along the generation request to. In this case the Vertex Endpoint will provide traffic sharing and versioning by handling the routing among multiple posssible backend models or instances while the docker container running the model will handle the actual request handling and prociessing.

Hence the need for both a model running framework (i.e. the actual loading of the model weights and computation) as well as a model serving framework (i.e. the web server and request processing). In this example we will use pytorch and VLLM.

Model running frameworks
========================

Model running frameworks provide a variety of options that try to acheive different goals

* pytorch
* HuggingFace Transformers
* HuggingFace Accelerate
* DeepSpeed Inference
* Nvidia TensorRT-LLM
* FasterTransformer

Model Serving Frameworks
========================

Model serving frameworks handle both request level management (e.g. batching and allocating requests among different works) as well as tasks like real time streaming of generation. While outside the scope of this example, they are also able to do things such as efficient request level switching between different fine-tuned variants of a model using techniques like LoRA, watermarking requests, or integration into telemetry infrastructure like Prometheus. While some frameworks like Triton are designed to work with a variety of different model running frameworks, others are tightly coupled to a particular framework.

Open source options include:
* FastAPI
* DJL-Serving [https://github.com/deepjavalibrary/djl-serving], Apache 2.0 license
* NVIDIA Triton [https://github.com/triton-inference-server/server], BSD-3 license
* vLLM [https://github.com/vllm-project/vllm], Apache 2.0 license
* HunggingFace Text Generation Inference (TGI) [https://github.com/huggingface/text-generation-inference], HFOILv1.0 license. 

Pytorch and vLLM on GCP
=======================

We will use pytorch and vLLM for this example. vLLM is designed to provide efficient serving of multiple models in production settings so this should provide a good starting point for future configuration and testing what an actual production deployment might look like. It provides an OpenAI api compatible server with streaming and support for many popular models since it leverages HF Transformers, although because it requires perfomance tweaks not all models are supported out of the box [https://docs.vllm.ai/en/latest/models/supported_models.html]. A description of the various arguments is here [https://docs.vllm.ai/en/latest/models/engine_args.html].

The first cell defiens the various constants to use. The model id and instance information is easily adjustable to try out different options. We leverage the built in docker container from vertex, it is possible to instead build and upload an image to GCP for your own needs.

The next cell defines the deployment function. It creates the endpoint, uploads the docker container to create the model and then deploys the model onto the endpoint. The last cell calls it with the parameters from the first cell. Note that deploying can take a while.

In [ ]:
# This is the name of the model on hugging face. When the docker container
# launches on vertex AI it will download and start this up
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# The g2 instance come with built in GPUs support
# The nvidia L4 is and Ada GPU that comes with 24GB of memory and 7,424 CUDA cores and 232 Tensor cores

machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Docker image to be used for serving the model. We will use vertexAI provided vllm server to host the models
VLLM_DOCKER_IMAGE = (
    "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve"
)

In [ ]:
from datetime import datetime
import os
from typing import Tuple

from google.cloud import aiplatform


def get_job_name_with_datetime(prefix: str):
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,  # we will need to set the service account for use later
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys Mistral models with vLLM on Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint", labels=GCP_TAGS)

    dtype = "bfloat16"
    if accelerator_type in ["NVIDIA_TESLA_T4", "NVIDIA_TESLA_V100"]:
        dtype = "float16"

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--dtype={dtype}",
        "--gpu-memory-utilization=0.9",
        "--max-num-batched-tokens=8192",  # trim the context length to work with our instance
        "--max-model-len=8192",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_IMAGE,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",  # route that predicts will be passed to
        serving_container_health_route="/ping",  # route used by vertex AI to verify the server is
        labels=GCP_TAGS,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
        min_replica_count=1,  # autoscale to zero is not currently supported by GCP so please terminate your instance
    )
    return model, endpoint

In [ ]:
# Creating and deploying the endpoint will take some time (e.g. around 20 min)
model_without_peft, endpoint_without_peft = deploy_model_vllm(
    model_name=get_job_name_with_datetime(prefix="mistral-serve"),
    model_id=model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)

print("endpoint_name:", endpoint_without_peft.name)

Creating Endpoint
Create Endpoint backing LRO: projects/948912860068/locations/us-central1/endpoints/34949076600487936/operations/8008735975373012992


Endpoint created. Resource name: projects/948912860068/locations/us-central1/endpoints/34949076600487936
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/948912860068/locations/us-central1/endpoints/34949076600487936')
Creating Model
Create Model backing LRO: projects/948912860068/locations/us-central1/models/3642125669938233344/operations/3111071380607598592
Model created. Resource name: projects/948912860068/locations/us-central1/models/3642125669938233344@1
To use this Model in another session:
model = aiplatform.Model('projects/948912860068/locations/us-central1/models/3642125669938233344@1')
Deploying model to Endpoint : projects/948912860068/locations/us-central1/endpoints/34949076600487936
Deploy Endpoint model backing LRO: projects/948912860068/locations/us-central1/endpoints/34949076600487936/operations/2167567258673479680


Endpoint model deployed. Resource name: projects/948912860068/locations/us-central1/endpoints/34949076600487936
endpoint_name: 34949076600487936


The endpoint created is not public. Often a service is put in front to handle public requests, e.g. using cloud functions[https://cloud.google.com/functions] to authenticate the user request and then calling the endpoint. For services running in Goggle cloud on the same server account, Google will seamlessly hand the authentication. Since this examples uses a DataRobot deployment, any such logic can be added there so it is not include. Since the aiplatform is initalized with appropriate credentials it can just be called with that. Vertex AI expects that requests are in an `instances` array, it will take each element and pass that along to the model. It will then return the values from the endpoint in a `predictions` array.

For example as json:

```
{
instances: [{...}]
}

{
predictions: [{...}]
}
```

Since the endpoint is hitting the /generate route which is not OpenAI compatible, the input format is defined here: [https://github.com/vllm-project/vllm/blob/main/vllm/entrypoints/api_server.py].

```
    The request should be a JSON object with the following fields:
    - prompt: the prompt to use for the generation.
    - stream: whether to stream the results or not.
    - other fields: the sampling parameters (See `SamplingParams` for details).`
```

The reponse will be a single JSON dict containing the key: `text` and the combined prompt + generated text as the value.

In [ ]:
instances = [
    {
        "prompt": "<s><INST>How can bagging trees boost my Random Forest?</INST>",
        "n": 1,
        "max_tokens": 200,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_without_peft.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
<s><INST>How can bagging trees boost my Random Forest?</INST>
Output:
 Bagging, or Bootstrap Aggregating, is a method used to improve the performance and reduce overfitting of ensemble learning models such as Random Forests. Here's how it works:
1. **Bootstrap samples**: Each tree in the Random Forest is built on a different bootstrap sample of the training data, drawn with replacement. This means that each tree will see a subset of the original training data, with some instances possibly being included multiple times.
2. **Reducing variance**: By training each tree on slightly different data, Bagging helps to decrease the variance of the Random Forest. This is important because trees are prone to overfitting on the training data. When each tree is exposed to different instances in the training set, it will learn slightly different decision surfaces, and the errors will average out, resulting in better generalization performance.
3. **Diversity**: Bagging also enhances the dive

The endpoint URL can be called directly via REST. Since this a python environment, `requests` is a common library for making REST calls. To do so the authorization JWT token is included in the header. Since in this situation there are generated from personal credentials they will be short lived. To do so, the Google authentication libraries provide helper utilities to create a `requests.Session` object that manages adding the authentication token to the request.

If running locally, e.g. using `curl` the `gcloud` CLI can generate the tokens. To call via a subprocess invoke the following:
```
subprocess.run(["gcloud", "auth", "print-access-token"], capture_output=True).stdout.decode().removesuffix("\n")
```
Or to invoke in the terminal:

```
gcloud auth print-access-token
```


In [ ]:
endpoint_url = f"https://{endpoint_without_peft.location}-aiplatform.googleapis.com/v1/{endpoint_without_peft.resource_name}"

In [ ]:
from google.auth.transport.requests import AuthorizedSession

# We will grab the acces token from the command line for our use to stick in the request headers
# Note that for curl the command looks like
# curl \
# -X POST \
# -H "Authorization: Bearer $(gcloud auth print-access-token)" \
# -H "Content-Type: application/json" \
# https://...

headers = {
    "Content-Type": "application/json",
}
authed_session = AuthorizedSession(credentials)  # This creates a request.Session object

response = authed_session.post(
    endpoint_url + ":predict", json={"instances": instances}, headers=headers
)
assert response.status_code == 200, response.json()
print(response.json()["predictions"][0])

Prompt:
<s><INST>How can bagging trees boost my Random Forest?</INST>
Output:
 Bagging, or Bootstrap Aggregating, is a technique used to improve the performance and stability of machine learning models, including Random Forests, by creating multiple base models from different sub-samples of the original training dataset. Here's how it can boost Random Forest:
1. Diversity: Bagging reduces overfitting by creating several Decision Trees and combining their outputs to increase the diversity of the model. By training trees on different samples of the data, Bagging helps ensure that the trees learn different aspects of the data, which in turn helps to reduce the correlation among the trees in the Random Forest.
2. Reducing variance: Bagging reduces the variance of the model by averaging the predictions of individual Decision Trees. Since each tree is trained on a different bootstrap sample, the errors made by one tree may be different from the errors made by another tree. By combining the p

 Since the URL is not public by default, it returns 401 unauthorized without a proper token

In [ ]:
import requests

response = requests.post(
    endpoint_url + ":predict",
    json={"instances": instances},
    headers={"Content-Type": "application/json"},
)
assert response.status_code == 401
response.json()

{'error': {'code': 401,
  'message': 'Request is missing required authentication credential. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.',
  'status': 'UNAUTHENTICATED',
  'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo',
    'reason': 'CREDENTIALS_MISSING',
    'domain': 'googleapis.com',
    'metadata': {'method': 'google.cloud.aiplatform.v1.PredictionService.Predict',
     'service': 'aiplatform.googleapis.com'}}]}}

Creating a DataRobot Custom model deployment
==================================

To bring this into DataRobot we can create a custom model. This will us to setup all the monitoring goodness of DataRobot around this model.

Because we would like to be able to have the deployment and be shareable without sharing our own private credentials (like we used above to interact with the deployment) we will use theservice account. 

For local development we can impersonate the service account by using our existing credentials to generate service account credentials. This way we can validate that the service account is properly set up and has all the appropriate permissions.



Then when we run as a deployment we can use Credential Management to hold the service account's private key. By doing this, the service account can then generate tokens using the private key for as long as the private key is valid. Also, by using a service account, on the google side it is no longer tied to the individual account and can be revoked separately. Lastly, if the key is compromised, the service account will only have the privileges granted it which are typically far less than a developer.

In [ ]:
# Redefine the constants from above so that can start right here with the session without redeploying on GCP
SERVICE_ACCOUNT = "octo-experiments-service-accou@octo-385122.iam.gserviceaccount.com"

# GCP Credential id from credential management with the uploaded private key
DR_GCP_CREDENTIAL_ID = "658c77e4ad86cd2ebecc9d7e"

# DR Prediction enviornment
DR_PREDICTION_ENVIRONMENT_ID = "5f06612df1740600260aca72"


custom_dir = "mistral_gcp"
# api_token = subprocess.run(["./gcloud", "auth", "print-access-token", f"--impersonate-service-account={SERVICE_ACCOUNT}"], capture_output=True).stdout.decode().removesuffix("\n")

# Here are the default runtime parameters we will need to set
LOCATION = REGION  # or "us-central1"
NUMERIC_PROJECT_ID = "948912860068"
ENDPOINT_ID = endpoint_without_peft.name
MAX_TOKENS = "1000"
TEMPERATURE = "1.0"
TOP_P = "1.0"
TOP_K = "10"

# This will be the prompt template that we wrap the prompt with. Mostly to handle the special tokens etc.
SYSTEM_PROMPT_TEMPLATE = "<s><INST>You are a helpful AI assistant, created by a French AI company. Be helpful and complete but also strive for concision. {}</INST> "

As is typical for custom inference models, we will need to provide a model-metadata.yaml and a custom.py. The existing GenAI builtin environment has the google cloud libraries built in along with the other libraries used so a requirements.txt is not required. In the model metadata, the target type is set as `textgeneration` to have access to text generation specific monitoring capabilities and the various runtime parameters are set as options. Runtime parameters allow easy updating of the DataRobot model endpoint if needed as part of a governed upgrade version. By changing the endpoint parameters and creating a new version, it is then possible to compare an older model with a newer version, if an update is required. It also allows configuration of the temperature etc. as part of the deployment and exposes these settings as metadata within DataRobot for reference. 

In [ ]:
model_metadata = """
name: gcp-vertex-proxy
type: inference
targetType: textgeneration

runtimeParameterDefinitions:
  - fieldName: location
    type: string
    defaultValue: us-central1
    description: The GCP location the endpoint resides in.

  - fieldName: gcp_credentials
    type: credential
    credentialtype: gcp
    description: The GCP service account key. This will be set to use the credential id set in credential manageament
    
  - fieldName: projectId
    type: string
    description: Numeric GCP projet for the endpoint
  
  - fieldName: endpointId
    type: string
    description: Numeric Vertex AI endpoint id

  - fieldName: maxTokens
    type: string
    defaultValue: "200"
    description: Maximum number of tokens to return
    
  - fieldName: temperature
    type: string
    defaultValue: "1.0"
    description: temperature of the model

  - fieldName: promptTemplate
    type: string
    description: promptTemplate for the model. It should contain {} which will be filled in with the user prompt

  - fieldName: topP
    type: string
    defaultValue: "1.0"
    description: Top p for the model
    
  - fieldName: topK
    type: string
    defaultValue: "10"
    description: Top k for the model

"""

And here is the custom py file written as text.

In the load model function we load the runtime parameters and verify that we can connect to the endpoint. It will prefer the passed token over the GCP service account credentials if provided. 

It will then take the user provide prompt, inject it into the promptTemplate with a format and then make a prediction. Lastly, because Mistral returns the prompt in the response, the original prompt is stripped out. Because the custom model API provides the data as pandas DataFrame into the `score` function and expects the output as a DataFrame as well, this is all run in a loop over the dataframe, causing each row of the DataFrame to be processed as a separate request. 

The prompt column is set to `prompt` to match the name that will be set when the model is registercd with the playground. The generated text column is set to `response` and will be registered as the `target name` when the DR Deployment is created.

Lastly, this and the model metadata yaml are then written to the filesystem inside a directory for uploading to datarobot.

In [ ]:
custom_py = """
import os
import base64
import json
import logging
import ssl
from types import SimpleNamespace
from typing import Any, Dict

import pandas as pd

from datarobot_drum import RuntimeParameters
from google.cloud import aiplatform
from google.oauth2 import service_account

import google.auth.credentials
from google.auth.transport.urllib3 import AuthorizedHttp
import google.oauth2.credentials

logger = logging.getLogger(__name__)

def _test_connectivity(location, project, endpoint_id, creds):
    # The root path of the endpoint can be used for health checks
    url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/endpoints/{endpoint_id}"
    authed_http = AuthorizedHttp(creds)  # Implements a urllib.RequestsMethods like a PoolManager
    try:
        authed_http.request('GET', url=url)
    except urllib.error.HTTPError as error:
        logger.error(
            "Failed to connect to %s status_code=%s\\n%s",
            url,
            error.code,
            error.read().decode("utf8", "ignore"),
        )
        raise


def load_model(code_dir):
    ''' Load runtime parameters and verify the endpoint is up'''
    logger.info("Loading Runtime Parameters...")
    cred_parameter = RuntimeParameters.get('gcp_credentials')['gcpKey']

    endpoint_id = RuntimeParameters.get("endpointId")
    location = RuntimeParameters.get("location")
    project = RuntimeParameters.get("projectId")
    prompt_template = RuntimeParameters.get("promptTemplate")
    max_tokens = int(RuntimeParameters.get("maxTokens"))
    temperature = float(RuntimeParameters.get("temperature"))
    top_p = float(RuntimeParameters.get("topP"))
    top_k = int(RuntimeParameters.get("topK"))
    
    gcp_token = os.getenv('GOOGLE_ENCODED_SERVICE_ACCOUNT_TOKEN', None)
    if gcp_token:
        # Handle case user provided base64 encoded credentials, e.g. in a notebook
        creds = google.oauth2.credentials.Credentials(token=gcp_token)
    else:
        # Handle case it pulled the values from the runtime parameters storage
        creds = service_account.Credentials.from_service_account_info(cred_parameter)
        creds = creds.with_scopes(['https://www.googleapis.com/auth/cloud-platform'])
    _test_connectivity(location, project, endpoint_id, creds)

    # Can return any object as a placeholder for a model that we can
    # then use again in the `score()` function.
    return SimpleNamespace(**locals())


def make_vertex_prediction(user_prompt, model):
    prompt = model.prompt_template.format(user_prompt)
    instances = [
    {
        "prompt": prompt,
        "n": 1,
        "max_tokens": model.max_tokens,
        "temperature": model.temperature,
        "top_p": model.top_p,
        "top_k": model.top_k,
    },
]
 
    endpoint = aiplatform.Endpoint(model.endpoint_id, project=model.project,
      location=model.location, credentials=model.creds)
    response = endpoint.predict(instances=instances)

    for prediction in response.predictions:
        # Remove everything from the response but the generated text
        out = prediction.partition("Output:\\n")[2]
    return out

def score(data, model, **kwargs):
    '''
    This hook is only needed if you would like to use **drum** with a framework not natively
    supported by the tool.

    Note: While best practice is to include the score hook, if the score hook is not present
    DataRobot will add a score hook and call the default predict method for the library
    See https://github.com/datarobot/datarobot-user-models#built-in-model-support for details

    This dummy implementation reverses all input text and returns.

    Parameters
    ----------
    data : is the dataframe to make predictions against.
    model : is the deserialized model loaded by **drum** or by `load_model`, if supplied
    kwargs : additional keyword arguments to the method
    Returns
    -------
    This method should return results as a dataframe with the following format:
      Text Generation: must have column with target, containing text data for each input row.
    '''
    data = list(data["prompt"])
    generated_responses = ["".join(make_vertex_prediction(inp, model)) for inp in data]
    result = pd.DataFrame({"response": generated_responses})
    return result

"""

In [ ]:
import os

try:
    os.listdir(custom_dir)
except:
    os.mkdir(custom_dir)
with open(f"{custom_dir}/model-metadata.yaml", "w") as f:
    f.write(model_metadata)

with open(f"{custom_dir}/custom.py", "w") as f:
    f.write(custom_py)

We can test locally using our local credentials and drum to make sure everything is working correctly. We will use our local credentials to create a token for the service account via a method call impersonation. Impersonation allows a user to use their credentials, the source, to then get the credentials of a different entity, the target. Because both the scopes, what services the token is valid for, and the permissions are able to be narrowly defined, these tokens are generally far less powerful than the source credentials.

Separately, this makes it easier to manage access across multiple accounts. As long as a user has access to the service account, they have access to the resources to make a prediction without worrying about needing to grant the appropriate permissions for each indiviudal resource, it also means that a user can be removed easily without breaking things (e.g. if the creator leaves the organization).

This code makes a subprocess call to the `drum` custom model utility and checks that it completed successful. The output predictions should be displayed.

In [ ]:
import subprocess

from google.auth import impersonated_credentials

runtimeparams = f"""
location: "{LOCATION}"
projectId: "{NUMERIC_PROJECT_ID}"
endpointId: "{ENDPOINT_ID}"
maxTokens: "{MAX_TOKENS}"
temperature: "{TEMPERATURE}"
topP: "{TOP_P}"
topK: "{TOP_K}"

promptTemplate: "{SYSTEM_PROMPT_TEMPLATE}"

gcp_credentials:
  credentialType: "gcp"
  gcpKey:  "FAKE_CREDS"
"""

input_csv = "prompt,\nHow soon is now?,\nWhat are the names of the Greek winds?,"

with open("mistral_values", "w") as f:
    f.write(runtimeparams)

with open("input.csv", "w") as f:
    f.write(input_csv)


target_scopes = ["https://www.googleapis.com/auth/cloud-platform"]
target_credentials = impersonated_credentials.Credentials(
    source_credentials=credentials,
    target_principal=SERVICE_ACCOUNT,
    target_scopes=target_scopes,
    lifetime=500,
)

# Call refresh to generate the token
target_credentials.refresh(request=google.auth.transport.requests.Request())

envs = os.environ
envs["GOOGLE_ENCODED_SERVICE_ACCOUNT_TOKEN"] = target_credentials.token
subprocess.check_call(
    [
        "drum",
        "score",
        "--code-dir",
        f"{custom_dir}",
        "--target-type",
        "textgeneration",
        "--input",
        "input.csv",
        "--runtime-params-file",
        "mistral_values",
    ],
    env=envs,
)

                                         Predictions
0  The phrase "How soon is now?" can be interpret...
1   Certainly, I'd be happy to help you with that...
0

Setting Runtime Parameters and Registering the Custom Model
===========================================================

Now that the it works locally it can be uploaded to DataRobot and added to the Model Registry. This will

* Execution Environment - The python enviornment in which the code will run. This will use the prebuilt `Python 3,11 GenAI` environment, the last prerequisite for creating the Custom Model
* Custom Inference Model - Created in the Custom Model Workshop with the appropriate metadata
* Custom Inference Model Vesion - A particular version of the environment, python code and runtime parameters under the Custom Inferene Model
* Custom Model Test -  Verification that the Custom Model works in the DataRobot environment
* Registered Model - Moved from the Workshop to the Registry. At this point appropriate governance policies can be applied
* Prediction Environment - Where the model is hosted if the account has multiple hosting options.

In [ ]:
import datarobot as dr

execution_environment = dr.ExecutionEnvironment.list("Python 3.11 GenAI")[0]

In [ ]:
# Patch to asllow TextGeneration target types in the python client.
dr.enums.CUSTOM_MODEL_TARGET_TYPE.ALL = dr.enums.CUSTOM_MODEL_TARGET_TYPE.ALL + ["TextGeneration"]

custom_model = dr.CustomInferenceModel.create(
    name="Mistral-7B Instruct on GCP",
    target_type="TextGeneration",
    target_name="response",  # Name as the output column we generate
    description="Mistral-7B proxy model",
    language="python",
)

After creating the specific model version, where we set that the container will have public network egress to be able to make calls over the internet.

The runtime parameters can be set via patching the REST API route until the datarobot sdk is updated to include setting runtime parameters.

In [ ]:
import json

model_version = dr.CustomModelVersion.create_clean(
    custom_model_id=custom_model.id,
    base_environment_id=execution_environment.id,
    folder_path=custom_dir,
    network_egress_policy=dr.NETWORK_EGRESS_POLICY.PUBLIC,  # need to be able to reach GCP over the public internet
)

url = model_version._path.format(model_version.custom_model_id)
path = f"{url}"
payload = {
    "baseEnvironmentId": execution_environment.id,
    "runtimeParameterValues": json.dumps(
        [
            {"fieldName": "temperature", "type": "string", "value": TEMPERATURE},
            {"fieldName": "location", "type": "string", "value": LOCATION},
            {"fieldName": "projectId", "type": "string", "value": NUMERIC_PROJECT_ID},
            {"fieldName": "endpointId", "type": "string", "value": ENDPOINT_ID},
            {"fieldName": "maxTokens", "type": "string", "value": MAX_TOKENS},
            {"fieldName": "topP", "type": "string", "value": TOP_P},
            {"fieldName": "topK", "type": "string", "value": TOP_K},
            {"fieldName": "promptTemplate", "type": "string", "value": SYSTEM_PROMPT_TEMPLATE},
            {"fieldName": "gcp_credentials", "type": "credential", "value": DR_GCP_CREDENTIAL_ID},
        ]
    ),
}
response = model_version._client.patch(path, json=payload)
model_version = dr.CustomModelVersion.get(custom_model.id, response.json()["id"])

To validste that everything works, there is a testing facility provided. This requires uploading a dataset to then pass into the test to verify it works. After uploading and asserting that it succeeded this uploaded dataset can then be deleted. 

In [ ]:
# Upload and create a dataset for testing custom model to validate it works correctly

test_dataset = dr.Dataset.create_from_file("input.csv")
test_dataset.id

'659711536646d197efceb6af'

In [ ]:
custom_test = dr.CustomModelTest.create(
    custom_model_id=custom_model.id,
    custom_model_version_id=model_version.id,
    dataset_id=test_dataset.id,
)
assert custom_test.overall_status == "succeeded", custom_test.get_log()

In [ ]:
# now delete the uploaded test dataset for the custom model test
test_dataset.delete(dataset_id=test_dataset.id)

Now that it has been verified to work, the model can be added to the registry and prepared for deployment

In [ ]:
registered_model = dr.RegisteredModelVersion.create_for_custom_model_version(model_version.id)

Creating and Testing the DR Deployment
======================================

From the registry it is strightforward to deploy given a prediction environment to deploy the model into.

Prediction Options

* Predict Batch: Predict using a Dataframe and the DataRobot client with the prompts as a column. The name of the column should match what we set in the custom model. This can score a large number of requests and can work in an async fashion.
* Realtime Predictions: Predict using JSON or CSV directly against the deployment in a synchronous fashion

To setup a synchronous prediction using either a cli like cURL or using the python requests library see the Predictions tab on the deployment and select Real-Time. 


In [ ]:
import pandas as pd

deployment = dr.Deployment.create_from_registered_model_version(
    registered_model.id,
    "Mistal 7B instruct on GCP",
    prediction_environment_id=DR_PREDICTION_ENVIRONMENT_ID,
)

prompt_df = pd.DataFrame({"prompt": ["How soon is now?", "What is your favorite wind?"]})
deployment.predict_batch(prompt_df)

Streaming DataFrame as CSV data to DataRobot
Created Batch Prediction job ID 659712c7bc8c7fd1baccff39
Waiting for DataRobot to start processing


Job has started processing at DataRobot. Streaming results.


,prompt,response_PREDICTION,DEPLOYMENT_APPROVAL_STATUS
0,How soon is now?,I'm an AI and don't have a concept of time or ...,APPROVED
1,What is your favorite wind?,"As a helpful and concise AI assistant, I don't...",APPROVED


Making the Deployment Available in the GenAI Playground
=======================================================

Using the datarobot-early-access client wwe can now register the model with the playground after it passes validation and then it will be available inside the playground to explore, compare against other models and hook up to grounding data in a Vector Database

In [ ]:
from datarobot._experimental.models.genai.custom_model_llm_validation import (
    CustomModelLLMValidation,
)

# If this doesn't work, try reinstalling or upgrading datarobot-early-accesss

custom_model_llm_validation = CustomModelLLMValidation.create(
    prompt_column_name="prompt",
    target_column_name="response",
    deployment_id=deployment.id,
    wait_for_completion=True,
)
assert custom_model_llm_validation.validation_status == "PASSED"

You have imported from the _experimental directory.
This directory is used for unreleased datarobot features.
Unless you specifically know better, you don't have the access to use this functionality in the app, so this code will not work.


Deleting Resources
================

To delete the created endpoint, set the following to true and then run the following.

In [ ]:
delete_dr_deployment = False
archive_dr_model_package = False
delete_dr_custom_model = False
delete_dr_llm_blueprint = False
delete_gcp_endpoint = False
delete_gcp_model = False


def delete_gcp_endpoint_by_id(endpoint_id):
    endpoint = aiplatform.Endpoint(endpoint_id, project=PROJECT_ID)
    endpoint.undeploy_all()
    endpoint.delete()


def delete_gcp_model_by_id(model_id):
    aiplatform.Model(model_id, project=PROJECT_ID).delete()


def delete_dr_deployment_by_id(deployment_id):
    dr.Deployment.get(deployment_id).delete()


def archive_dr_registered_model_by_id(registered_model_id):
    dr.RegisteredModel.get(registered_model_id).archive(registered_model_id)


def delete_dr_custom_model_by_id(custom_model_id):
    dr.CustomInferenceModel.get(custom_model_id).delete()


def delete_dr_llm_validation_by_id(genai_validation_id):
    dr._experimental.models.genai.custom_model_llm_validation.CustomModelLLMValidation.get(
        genai_validation_id
    ).delete()


if delete_gcp_endpoint:
    try:
        delete_gcp_endpoint_by_id(endpoint_without_peft.name)
        print(f"GCP Endpoint {endpoint_without_peft.name} deleted")
    except google.api_core.exceptions.NotFound:
        pass
if delete_gcp_model:
    try:
        delete_gcp_model_by_id(model_without_peft.name)
        print(f"GCP Model {model_without_peft.name} deleted")
    except google.api_core.exceptions.NotFound:
        pass
if delete_dr_deployment:
    try:
        delete_dr_deployment_by_id(deployment.id)
        print(f"DR Deployment {deployment.id} deleted")
    except dr.errors.ClientError as err:
        if not err.status_code == 404:
            raise
if archive_dr_model_package:
    try:
        archive_dr_registered_model_by_id(registered_model.model_id)
        print(f"DR Registered Model {registered_model.model_id} archived")
    except dr.errors.ClientError as err:
        if not err.status_code == 404:
            raise
if delete_dr_custom_model:
    try:
        delete_dr_custom_model_by_id(custom_model.id)
        print(f"DR Custom Model {custom_model.id} deleted")
    except dr.errors.ClientError as err:
        if not err.status_code == 404:
            raise
if delete_dr_llm_blueprint:
    try:
        delete_dr_llm_validation_by_id(custom_model_llm_validation.id)
        print(f"DR Custom Model LLM Blueprint {custom_model_llm_validation.id} deleted")
    except dr.errors.ClientError as err:
        if not err.status_code == 404:
            raise